# Модели текста и способы представления слов (word embeddings)

* "Глубокое обучение. Погружение в мир нейронных сетей" - Николенко, Кадурин, Архангельская
* [Word Embeddings by Lena Voita](https://lena-voita.github.io/nlp_course/word_embeddings.html)
* [Модель текста](https://ped_recheved.academic.ru/117/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0)
* [Автоматическая обработка текстов на естественном языке и анализ данных - Большакова, Воронцов, Ефремова, Клышинский, Лукашевич, Сапин](https://www.hse.ru/data/2017/08/12/1174382135/NLP_and_DA.pdf)

Модель текста - освобожденное от несущественных для целей моделирования представление содержания и формы текста (например, в опорных словах, словосочетаниях и предложениях), которое позволяет отобразить жанр, тему, композицию, структуру и способ развертывания содержания.

Для части задач компьютерной лингвистики не требуется полного понимания смысла текста, достаточно правильно описать его содержание. Модели текста позволяют сравнивать тексты друг с другом и единообразно обрабатывать их коллекции.

При этом можно выделить две группы подходов к построению моделей текста:
* Подходы, в которых не учитываются связи между словами (частотные или признаковые модели текста)
* Подходы, в которых связи учитываются (**дистрибутивная семантика**: информация о контекстах слов помещается в их векторное представление):
    * "Классические" статистические подходы: взаимная встречаемость слов зависит от глобальной статистики корпуса
    * Нейросетевые предиктивные подходы
    
Тогда мы можем представить слова как вектора вещественных чисел $R^d$: геометрические соотношения в пространстве $R^d$ будут соответствовать семантическим соотношениям между словами. Близкие векторы - слова-синонимы. 

**Дистрибутивная гипотеза**: лингвистические единицы, встречающиеся в схожих контекстах, имеют близкие значения. [Отличный пример от Лены Войты.](https://lena-voita.github.io/nlp_course/word_embeddings.html)

Рассмотрим различные примеры моделей текста.

## Признаковая модель текста

Признаковая модель рассматривает текст как неупорядоченный набор признаков:

* Лингвистические признаки
    * Слова
    * N-граммы
* Статистические признаки
    * Кол-во восклицательных знаков
    * Доля различных частей речи
* Экстралингвистические признаки
    * Заголовок документа
    * Дата публикации
    * Гиперссылки

Рассмотрим поподробнее модели, строящиеся на лингвистических признаках.

### Bag-of-Words (мешок слов)

Простейший пример текстовой модели можно реализовать при помощи **словаря разрешенных слов**, в котором все известные нам слова закодированы уникальными индексами.

In [1]:
vocabulary = {0: 'UNK', 1: 'i', 2: 'saw', 3: 'a', 4: 'cat'}

Пусть теперь у нас имеется $N$ текстов, мы можем представить каждый текст в виде вектора (**one-hot vectors**): $w_j = (w_{j1}, ..., w_{jM})$, где $w_{ji}$ - вес i-го слова в j-ом тексте, $M$ - размер словаря.

Таким образом, **мешок слов** - это модель представления текста в виде вектора (неупорядоченного набора слов). Каждому слову в тексте сопоставляется число его вхождений.

[Пример](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X.toarray(), vectorizer.get_feature_names()

(array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
        [0, 2, 0, 1, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 0, 0, 1, 0, 1]]),
 ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'])

Недостатки подхода:
* Не учитываются связи между словами и не учитывается порядок слов в тексте
    * Банк купил компанию и Компания купила банк - это одинаковые вектора
* Стоп-слова (предлоги, союзы и прочее) будут наиболее частотными в каждом документе, их стоит удалять
* Для больших словарей вектора будут очень длинными

### N-граммная модель текста

Данный подход похож на bag-of-words с той лишь разницей, что используются последовательности слов длины N. Таким образом, можно сказать, что мы учитываем порядок слов.

В данном подходе можно использовать не только слова, но и **буквенные N-граммы**.

In [3]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X = vectorizer.fit_transform(corpus)
X.toarray(), vectorizer.get_feature_names()

(array([[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]]),
 ['and this',
  'document is',
  'first document',
  'is the',
  'is this',
  'second document',
  'the first',
  'the second',
  'the third',
  'third one',
  'this document',
  'this is',
  'this the'])

### Skip-граммы

k-skip-n-граммы - наборы из n токенов, причем расстояние между соседними должно составлять не более k токенов.

In [4]:
from nltk.util import skipgrams

In [5]:
sent = "Insurgents killed in ongoing fighting".split()

In [6]:
list(skipgrams(sent, 2, 2))

[('Insurgents', 'killed'),
 ('Insurgents', 'in'),
 ('Insurgents', 'ongoing'),
 ('killed', 'in'),
 ('killed', 'ongoing'),
 ('killed', 'fighting'),
 ('in', 'ongoing'),
 ('in', 'fighting'),
 ('ongoing', 'fighting')]

In [7]:
list(skipgrams(sent, 3, 2))

[('Insurgents', 'killed', 'in'),
 ('Insurgents', 'killed', 'ongoing'),
 ('Insurgents', 'killed', 'fighting'),
 ('Insurgents', 'in', 'ongoing'),
 ('Insurgents', 'in', 'fighting'),
 ('Insurgents', 'ongoing', 'fighting'),
 ('killed', 'in', 'ongoing'),
 ('killed', 'in', 'fighting'),
 ('killed', 'ongoing', 'fighting'),
 ('in', 'ongoing', 'fighting')]

## Статистическая модель текста

### Positive Pointwise Mutual Information (PPMI)

Точечная взаимная информация (PMI) помогает ответить на вопрос: происходят ли события x и y совместно, чем по отдельности? Эту идею легко переложить на концепцию совстречаемости слов (Church & Hanks 1989) и рассчитать матрицу PPMI:

$$\text{PMI}(w_1, w_2) = \log_2{(\frac{P(w_1, w_2)}{P(w_1)P(w_2)})}$$

$$\text{PPMI}(w_1, w_2) = \text{max}(0, \text{PMI}(w_1, w_2))$$

### Матрица совместной встречаемости

Выше уже были предложены различные методы для формирования матрицы совместной встречаемости $ X \in R^{V \times V}$, где $V$ - набор токенов (слов, N-грамм и т.д.).

Один из наибольших недостатков матрицы совместной встречаемости - сильная разряженность.

Для эффективного хранения такой большой структуры можно использовать **[сингулярное разложение матриц (SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition)**: трансформируем матрицу в вектора.

> [Latent semantic analysis (LSA)](https://en.wikipedia.org/wiki/Latent_semantic_analysis)

LSA тесно связано с тематическим моделированием (pLSA), которое является его вероятностной интерпретацией.

Латентное размещение Дирихле (LDA) - байесовский вариант pLSA.

### TF-IDF (задание весов признакам)

**TF (term frequency — частота слова)** — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова $t_i$ в пределах отдельного документа:

$$\text{tf}(t, d) = \frac{n_t}{\sum\limits_{k} n_k}$$

где 
* $n_t$ - число вхождений слова $t$ в документ 
* $\sum\limits_{k} n_k$ - число слов в документе

**IDF (inverse document frequency — обратная частота документа)** — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Учёт IDF уменьшает вес широкоупотребительных слов. Для каждого уникального слова в пределах конкретной коллекции документов существует только одно значение IDF.

$$\text{idf}(t, D) = log(\frac{|D|}{|\{ d_i \in D | t \in d_i \}|})$$

где 
* $|D|$ - число документов в коллекции
* $|\{ d_i \in D | t \in d_i \}|$ - число документов в коллекции $D$, в которых встречается $t$ (когда $n_t \neq 0$)

Выбор основания логарифма в формуле не имеет значения, поскольку изменение основания приводит к изменению веса каждого слова на постоянный множитель, что не влияет на соотношение весов. Логарифм используется, чтобы «смягчить» разницу в употреблении более частотных и менее частотных слова.

Таким образом, мера TF-IDF является произведением двух сомножителей:

$$\text{tf-idf}(t, d, D) = \text{tf}(t, d) \times \text{idf}(t, D)$$

Большой вес в TF-IDF получат слова с высокой **частотой** в пределах конкретного документа и с низкой частотой употреблений в других документах.

Документы как вектора: 
* $|V|$ - мерное векторное пространство
* Слова – это отдельные позиции (оси) в векторах
* Документы – точки или вектора в пространстве
* Очень большой размерности: десятки миллионов  для интернета
* Вектора документов – разреженные

Формализация близости векторов:
* Вектора запроса и документов – точки в многомерном пространстве
* Как найти близость между векторами? Предположение: чем меньше угол, тем более похожи вектора. Косинусная мера: чем больше косинус  угла между векторами, тем более похожи документы

$$\cos(\vec q, \vec d) = \frac{\vec q \cdot \vec d}{|q| |d|} = \frac{\sum q_i d_i}{\sqrt{\sum q_i^2} \sqrt{\sum d_i^2}}$$

где
* $q_i$ - tf-idf вес слова i в запросе
* $d_i$ - tf-idf вес терма i в документе
* $q$ может быть не только запросом, но и другим документом

$\cos(\vec q, \vec d)$ - это косинусное сходство между $\vec q$ и $\vec d$, что эквивалентно косинусу угла между $\vec q$ и $\vec d$.

> [Tf-Idf and Cosine similarity](https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/)

Варианты весов
* Базовый – это сколько раз слово встретилось в документе (count), 
* Логарифмирование count
* Нормализация первого сомножителя (tf/tfmax или tf/|d|, где |d| - количество слов в документе)

TF-IDF опирается на предположение: **при обработке коллекции текстов частотные признаки менее информативны, чем редкие. Веса частотных признаков должны быть ниже, чем веса редких.**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X.shape, vectorizer.get_feature_names()

((4, 9),
 ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'])

## Предиктивная модель текста

### [Word2Vec (2013)](https://arxiv.org/pdf/1301.3781.pdf)

* [Word2Vec - wiki](https://en.wikipedia.org/wiki/Word2vec)
* [Learning Word Embedding](https://lilianweng.github.io/posts/2017-10-15-word-embedding/)
* [word2vec Parameter Learning Explained](https://arxiv.org/pdf/1411.2738.pdf)

Идея word embeddings заключается в том, чтобы каждому слову из словаря $i \in V$ сопоставить вектор $w_i \in R^d$, где $d$ - это достаточно большое значение, но все еще меньшее размера словаря (наша цель - довольно небольшая, но плотная матрица представлений). 

Вероятность того, что слово $i$ встречается в данном контексте (то есть вместе со словами $c_1, ..., c_n$): 

$$\hat p (i | c_1, ..., c_n) = f(w_i, w_{c_1}, ..., w_{c_n}; \theta)$$

Где $w_{c_1}, ..., w_{c_n}$ - это векторы слов контекста, $f$ - функция с обучаемыми параметрами $\theta$. Функция... Или нейронная сеть.

Будем обучать такую нейронную сеть на некотором корпусе/коллекции текстов, оптимизируя логарифм общего правдоподобия корпуса:

$$L(W, \theta) = \frac{1}{T} \sum_t \log{f(w_t, w_{t-1}, ..., w_{t-n+1}; \theta)}$$

где $T$ - количество возможных окон слов (3-10 окруждающих слов).

Какая архитектура может быть у такой нейронной сети? В статье [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf) исследователь Миколов предложил сразу 2 вида:

<img src="pictures/word2vec3.png" width=700 height=700 />

#### **CBOW (Continuous Bag-of-Words)**
* Предсказывает слово при заданном контексте (мешке слов). То есть мы знаем окружение слова справа и слева в некотором заданном окне, нужно непосредственно предсказать это слово.
* Архитектура - двухслойная нейронная сеть-кодировщик прямого распространения, которая достаточно быстро работает для больших корпусов. В упрощенном виде ее можно записать так:

```python
class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size=128):
        super(CBOW, self).__init__()

        # Слой 1
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_size)
        self.activation_function1 = nn.ReLU()
        
        # Слой 2: i-ая строка - представление i-го слова из словаря
        self.linear2 = nn.Linear(hidden_size, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)
        
    def forward(self, inputs):
        # Сумма или усреднее входных эмбеддингов (контекста)
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out
```

<img src="pictures/word2vec-cbow.png" width=500 height=500 />

#### **SkipGrams (Skip-n-Grams)**
* Предсказывает контекст при заданном слове, где контекст - это N-граммы
* Архитектура - также двухслойная нейронная, работающая медленне, чем для CBOW (эффективнее для поиска редких слов в небольших корпусах).

Подробнее с математическими аспектами обучения модели Word2Vec можно ознакомиться в статье: [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/pdf/1402.3722.pdf). Наиболее важными моментами являются:
* Для одного и того же слова используется 2 вектора: один - когда слово является центральным словом в окне (c - central), а второй - когда слово является одним из слов контекста (o - outside). 
    * Cлово может встретиться в своем же контексте! 
* Таким образом, мы **оптимизируем 2 матрицы параметров**, a соответствующая softmax-функция будет выглядеть следующим образом:

$$p(o|c) = \frac{\text{exp}(u_o^T v_c)}{\sum_{w \in V} \text{exp}(u_w^T v_c)}$$

* Таким образом, при вычислении градиента мы будем считать его для пары слов (текущее центральное слово, текущее слово контекста). Грубо говоря, мы бежим в цикле по словам из словаря и проверям, встречается ли слово контекста вместе с ним в некоторм окне. Тогда лосс-функция будет иметь вид:

$$-u_o^T v_c + \text{log}(\sum_{w \in V} \text{exp}(u_w^T v_c))$$

* **Negative sampling**
    * $\sum_{w \in V} \text{exp}(u_w^T v_c)$ - довольно сложная операция, нужно за один шаг обновить $|V| + 1$ параметр. Вместо того, чтобы считать полную сумму и обновлять все параметры, будем обновлять выборочно лишь некоторые из них. Так мы будем оптимизировать правдоподобие немного другого события, но это все еще будет работать.
    * Также в этом трюке мы переходим от softmax к сигмоиде: так как мы считаем градиент для пары слов, то можем поставить задачу **присутствия пары слов в словаре**, то есть бинарную классификацию. Сейчас у нас все пары из сканирующего окна в словаре есть, поэтому нужно насемплировать пары слов, которые в одном контексте не встречаютсе - нулевой класс.
    
> Можно показать, что Word2Vec неявно аппроксимируют факторизацию (сдвинутой) матрицы PMI (SVD)

Мы можем работать с [предобученным word2vec](https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300?select=GoogleNews-vectors-negative300.bin):

In [1]:
import gensim

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin',
    binary=True)

In [3]:
vocabulary = model.index_to_key
len(vocabulary)

3000000

In [4]:
model.most_similar('adorable')

[('cute', 0.7902224063873291),
 ('cutest', 0.648817777633667),
 ('lovable', 0.6460723876953125),
 ('adorable_puppy', 0.6357036828994751),
 ('loveable', 0.6204661726951599),
 ('sooo_cute', 0.6165933012962341),
 ('adorably', 0.6149011850357056),
 ('vulnerable_Frangipane', 0.6049544811248779),
 ('Adorable', 0.6045286059379578),
 ('endearing', 0.5991296172142029)]

Или можем обучить (дообучить) свою модель:

In [3]:
from gensim.models.word2vec import Word2Vec

In [5]:
model = Word2Vec(vector_size=300, window=7, min_count=10, workers=10)

In [7]:
model.build_vocab(corpus)

In [21]:
model.train(corpus, total_examples=len(corpus), epochs=3);

### [GloVe (Global Vectors - 2014)](https://aclanthology.org/D14-1162.pdf)

* [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)

GloVe - это объединение статистической и предиктивной модели: будем обучать градиентным спуском **глобальные статистики** корпуса текстов. 

Глобальные статистики до этого мы упомянали, когда говорили про матрицу совместной встречаемости слов: $X \in R^{V \times V}$. 

|  | мама | мыла | раму |
| --- | --- | --- | --- |
| мама | 0 | 6 | 0 |
| мыла | 0 | 1 | 3 |
| раму | 1 | 2 | 0 |

В такой матрице общее число совместных встречаемостей слова $i$: $X_i = \sum_k X_{ik}$, а вероятность встретить слово $j$ в контексте слова $i$:

$$p(j|i) = \frac{X_{ij}}{\sum_k X_{ik}}$$

Такие вероятности достаточно трудно интерпретировать в отличае от **отношений вероятностей**: чем больше значение соотношения, тем чаще одно слово встречается в контексте другого. Так GloVe обучает нейронную сеть, аппрксимирующую отношения вероятностей слов из глобального корпуса:

$$J(\theta) = \sum_{i, j = 1}^{|V|} f(X_{ij}) (w_i^T \tilde w_j + b_i + \tilde b_j - \log{X_{ij}})^2$$

где  
* $w_i$ - векторное представление слова, а $\tilde w_j$ - векторное представление контекста
* $b_i$ и $\tilde b_j$ - соответствующие векторы сдвига (bias)
* $f(x)$ - весовая функция, где $\alpha = 0.75$, а $x_{max} = 100$:
$$f(x)={\begin{cases}(\frac{x}{x_{max}})^\alpha,&\text{if } x < x_{max};\\1,& \text{otherwise}\end{cases}}$$

In [1]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [2]:
model.most_similar(positive=["cat", "adorable"], negative=["fluffy"])

[('dog', 0.6603151559829712),
 ('sister', 0.6491517424583435),
 ('cute', 0.6427398920059204),
 ('omg', 0.6382595300674438),
 ('friend', 0.6285562515258789),
 ('girl', 0.6278365850448608),
 ('animal', 0.6256914734840393),
 ('dad', 0.6187349557876587),
 ('picture', 0.6186913847923279),
 ('pet', 0.618313193321228)]

### [FastText (2017)](https://aclanthology.org/Q17-1010.pdf)

* [Library for efficient text classification and representation learning](https://fasttext.cc/)

Особенности:
* Negative sampling
* Эмбеддинги N-грамм
* Subword-embeddings модель: разбиение токена на символьные N-граммы по 3-6 символов:
    * Пример разбиения слова "проект" на буквенные триграммы: {'#пр', 'про', 'рое', 'оек', 'ект', 'кт#'}
    * Словарь из таких триграмм гораздо короче словаря всех слов
* Хэширование таблицы признаков
* Архитектура похожа на CBOW Word2Vec

> Deep Structured Semantic Models

## GPT Tokenizers

### Byte-Pair Encoding (BPE)

* [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)
* [BPEmb: Tokenization-free Pre-trained Subword Embeddings in 275 Languages](https://aclanthology.org/L18-1473.pdf)
* [Сайт проекта](https://bpemb.h-its.org/)
* [Репозиторий проекта](https://github.com/bheinzerling/bpemb)
* [Hugging face NLP course - Byte-Pair Encoding tokenization](https://huggingface.co/learn/nlp-course/en/chapter6/5)
* [Let's build the GPT Tokenizer - Andrej Karpathy](https://www.youtube.com/watch?v=zduSFxRajkE&ab_channel=AndrejKarpathy) + [repo - minbpe](https://github.com/karpathy/minbpe)
* [Токенизация на подслова (Subword Tokenization) - Александр Дьяконов](https://alexanderdyakonov.wordpress.com/2019/11/29/%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F-%D0%BD%D0%B0-%D0%BF%D0%BE%D0%B4%D1%81%D0%BB%D0%BE%D0%B2%D0%B0-subword-tokenization/)
* [tiktoken (a fast BPE tokeniser for use with OpenAI's models)](https://github.com/openai/tiktoken)

Впервые подход кодирования на подслова был предложен еще в 1994 (by Philip Gage). Суть подхода заключается в том, что в качестве токена будут использоваться не слова, а наборы символов с некоторой глубиной кодирования. Подход позволяет сжать последовательности за счет увеличения словаря токенов.

При помощи BPE-tokenizer можно решить несколько проблем, присутствующих у таких моделей как Word2Vec, Glove и т.п.:
* Независимость векторов для разных версий одного и того же слова
* Кодирование отсутствующих в словаре слов
* Большой размер словаря для больших корпусов текстов

Обучение BPE-алгоритма:
1. **Словарь токенов** инициализируется при помощи списка всех возможных символов, из которых состоят слова корпуса (как минимум, это все символы ASCII + некоторые символы Unicode)
2. Затем до достижения сходимости ищем в корпусе пары токенов, которые встречаются наиболее часто:
    * Инициализируем пустую таблицу (**merge table**)
    * Производим подсчет того, как часто пары символов встречаются в корпусе
    * Наиболее часто встречаемая пара символов добавляется в словарь и далее интерпретируется как **единый токен**

При инференсе BPE-алгоритма выученные правила применяются для сегментации слов:
* Каждое слово разбивается на последовательность символов
* Среди всех последовательностей символов ищем в merge table самую длинную и выполняем слияние

Таблица слияний упорядочена: слияния, находящиеся выше в таблице, чаще встречались в данных. Поэтому в алгоритме более высокие слияния имеют более высокий приоритет: на каждом шаге мы объединяем наиболее частое слияние среди всех возможных.

### Токенизаторы из LLM
* Токенизаторы GPT-2 и RoBERTa используют **byte-level BPE**: они рассматривают слова как байты (256 символов)
* В BERT используется **WordPiece**
* Другие токенизаторы:
    * **Unigram Language Model (ULM)**
    * **SentencePiece**
    * **BPE-Dropout**

## BACKLOG
* WordNet и Freebase
* RC-NET
* Doc2Vec
* PV-DM и PV-DBOW